# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv("../WeatherPy/output_data/weather_data.csv")
weather_data_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,punta arenas,-53.1500,-70.9167,42.8,93,40,3.44,CL,1619228976
1,busselton,-33.6500,115.3333,71.6,48,31,5.35,AU,1619228977
2,carnarvon,-24.8667,113.6333,73.4,43,2,8.05,AU,1619228978
3,hilo,19.7297,-155.0900,78.8,65,90,16.11,US,1619228777
4,puerto narino,-3.7703,-70.3831,75.2,100,40,2.30,CO,1619228980


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Set locations
locations = weather_data_df[["Lat", "Lng"]].astype(float)

# Define heatmap layer
humidity = weather_data_df["Humidity"].astype(float)

# Create gmaps figure
heatmap_fig = gmaps.figure()

# Create humidity heatmap layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity= 100,
                                 point_radius = 3)

# Add heatmap layer to gmaps figure
heatmap_fig.add_layer(heat_layer)

# Show heatmap
heatmap_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create dataframe withe ideal weather conditions
ideal_weather_df = weather_data_df[(weather_data_df['Max_Temp'] < 90) 
                                   & (weather_data_df['Max_Temp'] > 80) 
                                   & (weather_data_df['Wind_Speed'] < 10)
                                   & (weather_data_df['Cloudiness'] == 0)]                                         

# Show dataframe
ideal_weather_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
162,lalmohan,22.3384,90.7419,87.04,53,0,4.72,BD,1619229197
178,sharjah,25.3573,55.4033,80.60,46,0,4.61,AE,1619228990
250,taoudenni,22.6783,-3.9836,86.63,7,0,8.41,ML,1619229313
274,ormara,25.2088,64.6357,80.31,78,0,1.72,PK,1619229345
398,hermosillo,29.0667,-110.9667,82.00,28,0,5.01,MX,1619229508
455,mazagao,-0.1150,-51.2894,80.60,89,0,6.91,BR,1619229582
485,lasem,-6.6922,111.4527,82.26,69,0,5.01,ID,1619229626
507,thinadhoo,0.5333,72.9333,82.49,67,0,6.80,MV,1619229604
513,bambanglipuro,-7.9500,110.2833,82.40,67,0,3.76,ID,1619229661


In [5]:
# Count the amount of cities in Ideal Weather dataframe
ideal_weather_df['City'].count()

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create hotel dataframe
hotel_df = ideal_weather_df

# Add "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ""

# Show dataframe preview
hotel_df

<ipython-input-6-d42e5e2c2139>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
162,lalmohan,22.3384,90.7419,87.04,53,0,4.72,BD,1619229197,
178,sharjah,25.3573,55.4033,80.60,46,0,4.61,AE,1619228990,
250,taoudenni,22.6783,-3.9836,86.63,7,0,8.41,ML,1619229313,
274,ormara,25.2088,64.6357,80.31,78,0,1.72,PK,1619229345,
398,hermosillo,29.0667,-110.9667,82.00,28,0,5.01,MX,1619229508,
455,mazagao,-0.1150,-51.2894,80.60,89,0,6.91,BR,1619229582,
485,lasem,-6.6922,111.4527,82.26,69,0,5.01,ID,1619229626,
507,thinadhoo,0.5333,72.9333,82.49,67,0,6.80,MV,1619229604,
513,bambanglipuro,-7.9500,110.2833,82.40,67,0,3.76,ID,1619229661,


In [7]:
# Create looping API request to retrieve hotels and enter into hotel_df

# Set pause between API requests to 1 second
time_between = 1
 
# set up a parameters dictionary
params = {"radius": 5000,
          "type": "lodging",
          "keyword" : "hotel",
          "key": g_key}

# base url for API request
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop to retrieve hotel name and insert into hotel_df
for index, row in ideal_weather_df.iterrows():
    
    # Set latitude and longitude for each city
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the location params as previously set latitude and longitude
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (IndexError):
        
        # If there are no hotels retrieved enter 'NaN' into column
        hotel_df.loc[index, "Hotel Name"] = ["NaN"]
        
        # Pause between requests
        time.sleep(time_between)

# Show dataframe preview with inserted retrieved hotel names
hotel_df

C:\Users\alever\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\alever\Anaconda3\lib\site-packages\pandas\core\indexing.py:1760: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
162,lalmohan,22.3384,90.7419,87.04,53,0,4.72,BD,1619229197,Arafat Hotel
178,sharjah,25.3573,55.4033,80.60,46,0,4.61,AE,1619228990,Occidental Sharjah Grand
250,taoudenni,22.6783,-3.9836,86.63,7,0,8.41,ML,1619229313,NaN
274,ormara,25.2088,64.6357,80.31,78,0,1.72,PK,1619229345,Alqadri hotel zero point ormara
398,hermosillo,29.0667,-110.9667,82.00,28,0,5.01,MX,1619229508,Marsella 45
455,mazagao,-0.1150,-51.2894,80.60,89,0,6.91,BR,1619229582,NaN
485,lasem,-6.6922,111.4527,82.26,69,0,5.01,ID,1619229626,NaN
507,thinadhoo,0.5333,72.9333,82.49,67,0,6.80,MV,1619229604,NaN
513,bambanglipuro,-7.9500,110.2833,82.40,67,0,3.76,ID,1619229661,Puri Brata


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
heatmap_fig.add_layer(markers)

# Display figure
heatmap_fig

Figure(layout=FigureLayout(height='420px'))